In [0]:
# # 05 - KPI Report: Evaluación y Visualización

# COMMAND ----------

# IMPORTACIONES
from pyspark.sql.functions import col, count, when, isnan
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import roc_auc_score
import joblib

# CONFIGURACIÓN DE GRÁFICAS
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# COMMAND ----------

# 📌 Cargar tabla original y tabla limpia
df_raw = spark.table("datalottery.lotterybets.lottery_bets_dirty")
df_clean = spark.table("datalottery.lotterybets.lottery_bets_dirty_cleaned")

print(f"📊 Registros originales: {df_raw.count()}")
print(f"📈 Registros luego de limpieza: {df_clean.count()}")

# COMMAND ----------

# 🔍 Calcular nulos por columna en raw
from pyspark.sql.types import NumericType

# Separar columnas numéricas y no numéricas
numeric_cols = [f.name for f in df_raw.schema.fields if isinstance(f.dataType, NumericType)]
non_numeric_cols = [f.name for f in df_raw.schema.fields if not isinstance(f.dataType, NumericType)]

# Contar nulos en numéricas (isNull + isnan)
nulos_numeric = df_raw.select([
    count(when(col(c).isNull() | isnan(col(c)), c)).alias(c)
    for c in numeric_cols
])

# Contar nulos en no numéricas (solo isNull)
nulos_non_numeric = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in non_numeric_cols
])

# Unir y convertir a Pandas
nulos_raw = nulos_numeric.join(nulos_non_numeric).toPandas().T.rename(columns={0: 'nulos_raw'})

# 🔍 Calcular nulos por columna en clean
from pyspark.sql.types import NumericType

# Separar columnas numéricas y no numéricas
numeric_cols_clean = [f.name for f in df_clean.schema.fields if isinstance(f.dataType, NumericType)]
non_numeric_cols_clean = [f.name for f in df_clean.schema.fields if not isinstance(f.dataType, NumericType)]

# Contar nulos en numéricas (isNull + isnan)
nulos_numeric_clean = df_clean.select([
    count(when(col(c).isNull() | isnan(col(c)), c)).alias(c)
    for c in numeric_cols_clean
])

# Contar nulos en no numéricas (solo isNull)
nulos_non_numeric_clean = df_clean.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in non_numeric_cols_clean
])

# Unir y convertir a Pandas
nulos_clean = nulos_numeric_clean.join(nulos_non_numeric_clean).toPandas().T.rename(columns={0: 'nulos_clean'})

# Unir y mostrar
nulos_comparativo = pd.concat([nulos_raw, nulos_clean], axis=1)
nulos_comparativo["% reducción"] = (
    100 * (nulos_comparativo["nulos_raw"] - nulos_comparativo["nulos_clean"]) / nulos_comparativo["nulos_raw"].replace(0, 1)
)
display(nulos_comparativo)

# COMMAND ----------

# 📊 Gráfico: Comparativa de nulos por columna
nulos_comparativo_plot = nulos_comparativo.reset_index().rename(columns={"index": "columna"})
nulos_comparativo_plot = nulos_comparativo_plot[nulos_comparativo_plot["nulos_raw"] > 0]

plt.figure()
sns.barplot(data=nulos_comparativo_plot, x="columna", y="nulos_raw", color="red", label="Antes")
sns.barplot(data=nulos_comparativo_plot, x="columna", y="nulos_clean", color="green", label="Después")
plt.title("Comparación de valores nulos antes vs después de limpieza")
plt.ylabel("Nº de valores nulos")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# COMMAND ----------datalottery.lotterybets.lottery_bets_dirty

# 📊 Ver versiones de tabla Delta (Time Travel)
history_df = spark.sql("DESCRIBE HISTORY datalottery.lotterybets.lottery_bets_dirty_cleaned")
display(history_df.select("version", "timestamp", "operation", "operationMetrics"))

# COMMAND ----------

# 📌 Verificación: Tabla de features y modelo
df_features = spark.table("datalottery.lotterybets.lottery_bets_dirty_features")
print(f"✅ Features cargadas: {df_features.count()} registros")

# Convertir a pandas
df_pd = df_features.select(
    "bets_last_7d", "win_rate_last_30d", "ip_risk", "geo_risk", 
    "num_picks", "avg_stake_amount", "suspicious"
).toPandas()

# Separar X, y
X = df_pd.drop(columns=["suspicious"])
y = df_pd["suspicious"]

# Intentar cargar el modelo
try:
    model = joblib.load("/tmp/ProyectoMLOps_rf_model.joblib")
    y_pred = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, y_pred)
    print(f"🎯 AUC del modelo actual: {auc:.3f}")
except Exception as e:
    print("⚠️ No se pudo cargar el modelo. Error:", e)

# COMMAND ----------

# 📈 Gráfico: Distribución de variable objetivo
# Preparar DataFrame para gráfico barras lado a lado
nulos_comparativo_plot = nulos_comparativo.reset_index().rename(columns={"index": "columna"})
nulos_comparativo_plot = nulos_comparativo_plot[nulos_comparativo_plot["nulos_raw"] > 0]

# Convertir a formato largo
df_long = nulos_comparativo_plot.melt(
    id_vars='columna',
    value_vars=['nulos_raw', 'nulos_clean'],
    var_name='Estado',
    value_name='Nulos'
)

# Mapear nombres para leyenda
df_long['Estado'] = df_long['Estado'].map({'nulos_raw': 'Antes', 'nulos_clean': 'Después'})

# Graficar barras lado a lado con hue
plt.figure()
sns.barplot(data=df_long, x='columna', y='Nulos', hue='Estado')
plt.title("Comparación de valores nulos antes vs después de limpieza")
plt.ylabel("Nº de valores nulos")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# COMMAND ----------

# 🎯 KPI final resumen (puedes imprimir o exportar)

kpis = {
    "Registros originales": str(df_raw.count()),
    "Registros luego de limpieza": str(df_clean.count()),
    "Duplicados eliminados": str(df_raw.count() - df_clean.count()),
    "AUC del modelo (si cargó)": f"{auc:.3f}" if 'auc' in locals() else "N/A",
    "Columnas con nulos antes": str(int((nulos_comparativo["nulos_raw"] > 0).sum())),
    "Columnas con nulos después": str(int((nulos_comparativo["nulos_clean"] > 0).sum())),
    "Reducción total de nulos": str(int(nulos_comparativo["nulos_raw"].sum() - nulos_comparativo["nulos_clean"].sum())),
    "Versiones de la tabla Delta": str(history_df.count())
}

kpis_df = pd.DataFrame.from_dict(kpis, orient="index", columns=["Valor"])
display(kpis_df)